##  ISUZU IntelliChat
An intelliChat, conversational AI chatbot for enhanced customer engagement and support powered by Large Language Models (LLMs).

### Project Goal
To develop a state-of-the-art chatbot tailored for ISUZU that can assist with customer queries, vehicle recommendations, service scheduling, troubleshooting, and general brand interaction. The chatbot will be deployed steamlit for agents interactions.

### Project Flow
#### Problem Definition
<b> Objective </b>: Build an intuitive chatbot that aligns with ISUZU’s brand values: reliability, innovation, and customer satisfaction.

<i>Primary Use Cases</i>:
* <i> Vehicle Recommendations</i>: Help associates recommend the right ISUZU vehicle based on customers preferences and budget.
* <i> Customer Support</i>: Answer FAQs about vehicles, maintenance, and services.
* <i> Service Scheduling</i>: Assist Associates in booking vehicle services or repairs.
* <i> Dealer Locator</i>: Help customers find the nearest dealerships or service centers.
* <i> Lead Generation</i>: Collect potential customer information for follow-ups.

### Dataset Preparation
#### Data Sources:

<ol>
<li>ISUZU’s existing customer FAQs</li>
<li> training and support documents </li>
<li>Vehicle specification sheets</li>
<li>Brochures, and catalogs</li>
<li>Customer interaction logs (emails, chats, calls)</li>
<li>ISUZU’s website and marketing materials</li>
<li>Feedback from customer surveys</li>
</ol>


### Data Processing:

<ol>
<li> Clean and standardize textual data</li>
<li> Annotate intents (e.g., “service booking,” “vehicle specs query”)</li>
<li> Categorize entities like vehicle models, features, locations, and service types</li>
</ol>

### Data injection_____

### Model Selection
##### LLM Selection:
*  OpenAI GPT-4 or Anthropic Claude for their ability to handle complex and domain-specific conversations.
*  Hugging Face BLOOM or Meta’s LLaMA for open-source options if cost-efficiency is a priority.

<b>Fine-Tuning </b>:

Fine-tune the LLM on ISUZU-specific data to ensure domain expertise.
Use transfer learning to teach the model vehicle-related terminology, common queries, and ISUZU’s brand tone.


### Conversational Flow Design
##### Key Components:
* Intents: Vehicle recommendations, service booking, troubleshooting, etc.
* Entities: Vehicle models, features, service center locations, dates, times.
#### Dialogue Flow:
* User: “Which ISUZU vehicle is best for off-roading?”
* Chatbot: “The ISUZU D-Max is an excellent choice for off-roading, thanks to its advanced 4WD system and rugged build. Would you like to learn more about its features?”



##### Loading the libraries 

In [1]:
import os
import streamlit as st
import pickle
import time
import faiss

from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from llama_index.core import SimpleDirectoryReader
from langchain_core.documents import Document  # Import LangChain's Document class

#### 🔹 Set OpenAI API Key

In [3]:
from dotenv import load_dotenv
import os
OPENAI_API_KEY: str= os.getenv("OPENAI_API_KEY")


### 🔹 Initialize OpenAI LLM

In [4]:
# 🔹 Initialize OpenAI LLM
llm = OpenAI(temperature=0.9, max_tokens=500)

C:\Users\Thomas.Okiwi\AppData\Local\Temp\ipykernel_16208\2312513075.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9, max_tokens=500)


### 🔹 Loading Documents 

In [5]:
# 🔹 Load documents from the directory using LlamaIndex
folder_path = r"C:\Users\Thomas.Okiwi\OneDrive - Techno Brain Group\Documents\Data Science Projects\Generative AI\IsuuBot\Data"
llama_documents = SimpleDirectoryReader(folder_path).load_data()

Failed to load file C:\Users\Thomas.Okiwi\OneDrive - Techno Brain Group\Documents\Data Science Projects\Generative AI\IsuuBot\Data\Body price.csv with error: 'utf-8' codec can't decode byte 0x96 in position 79: invalid start byte. Skipping...


Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)


### 🔹 Convert LlamaIndex documents to LangChain-compatible format

In [6]:
# 🔹 Convert LlamaIndex documents to LangChain-compatible format with metadata
langchain_documents = []
for doc in llama_documents:
    langchain_documents.append(Document(page_content=doc.text, metadata={"source": doc.doc_id}))

### 🔹 Create a text splitter

In [7]:
# 🔹 Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)

In [8]:
# 🔹 Split the documents into chunks
split_docs = text_splitter.split_documents(langchain_documents)

# Print the number of chunks
print(f"✅ Total Chunks Created: {len(split_docs)}")
print(f"✅ Total Documents Loaded: {len(llama_documents)}")


✅ Total Chunks Created: 1165
✅ Total Documents Loaded: 499


### 🔹 Create Embeddings

In [9]:
# 🔹 Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# 🔹 Create FAISS Vector Store
vector_store = FAISS.from_documents(split_docs, embeddings)

# 🔹 Save FAISS Index
faiss_index_path = "vector_index.faiss"
faiss.write_index(vector_store.index, faiss_index_path)
print("✅ FAISS Index Saved Successfully!")

C:\Users\Thomas.Okiwi\AppData\Local\Temp\ipykernel_16208\846761174.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


✅ FAISS Index Saved Successfully!


In [10]:
# 🔹 Load the FAISS index
index = faiss.read_index(faiss_index_path)

# Check if the index is loaded correctly
if index.is_trained:
    print("✅ FAISS Index Loaded Successfully!")
else:
    print("❌ FAISS Index Loading Failed!")
    

✅ FAISS Index Loaded Successfully!


### 🔹 Creating a Retriever

In [11]:
# 🔹 Wrap FAISS with LangChain's retriever
retriever = vector_store.as_retriever()

# 🔹 Create the RetrievalQA Chain
qa_chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever)


### 🔹 Queries

In [12]:
# 🔹 Example Queries
example_queries = [
    " How much is the deposit and the monthly payment ? "
  
]


# 🔹 Run example queries
for query in example_queries:
    print("\n🔹 Query:", query)
    response = qa_chain({"question": query})
    print("🔹 Answer:", response["answer"])
    print("🔹 Sources:", response.get("sources", "No sources found"))



🔹 Query:  How much is the deposit and the monthly payment ? 


C:\Users\Thomas.Okiwi\AppData\Local\Temp\ipykernel_16208\91349922.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": query})


🔹 Answer:  The deposit is 5% of the vehicle price and it can change depending on the amount of financing you will get. The monthly payment information is not specified.

🔹 Sources: 8406bf16-3ccb-407c-b164-15180ff4404c


### Model Deployment 

In [13]:
## http://localhost:8501/